<a href="https://colab.research.google.com/github/FullteaOfEEIC/er/blob/master/er.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#seq2seq
!git clone https://www.github.com/farizrahman4u/recurrentshop.git
%cd recurrentshop
!python setup.py install
!pip install git+https://github.com/farizrahman4u/seq2seq.git
from seq2seq.models import SimpleSeq2Seq,AttentionSeq2Seq
import keras

#mecab
!apt install aptitude
!apt-get install swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3
import MeCab
%cd
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n

!pip install gensim
import gensim
from gensim.models import word2vec


#Google Drive
#Notes. Colaboratory may requires you a permission to access Google Drive.
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/
%pwd


import numpy as np
import matplotlib.pylab as plt
import math
import re
%matplotlib inline
import copy

In [0]:
!echo `mecab-config --dicdir`"/mecab-ipadic-neologd"
m=MeCab.Tagger("-d  /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd  -Owakati")

In [0]:
model_wv = word2vec.Word2Vec.load("./model/wiki.model")

In [0]:
url=re.compile(r"http(s)?://([\w\-]+\.)+[\w\-]+(\/[\w\- ./?%&=]*)?")
def line2vec(text,length=20):
  #記号類
  text=re.sub("[！!]+","!",text)
  text=re.sub("[？?]+","?",text)
  text=re.sub("\d+","0",text)
  text=text.lower()
  text=re.sub("w+","w",text)
  text=url.sub("",text)
  text=text.replace("[ファイル]","")
  
  #OLK特有の固有名詞、他
  text=text.replace("olk","オリエンテーリング サークル")
  text=text.replace("メンブレ","メンタル ブレイク")
  text=text.replace("ブリケ","ストレージ")
  text=re.sub("オリエンテーリング|オリエン","オリエンテーリング",text)
  text=text.replace("スクショ","スクリーンショット")
  text=text.replace("追いコン","追い出し コンパ")
  
  #人名#この置換処理は個人情報保護の観点から記載していません。

  
  
  parsed=m.parse(text).strip().split(" ")
  
  vector=[model_wv[p] for p in parsed if p in model_wv]
  if length:
    if length<len(vector):
      vector=vector[len(vector)-length::]
    else:
      for _ in range(length-len(vector)):
        vector.append(model_wv["。"])
  return vector
  
def vec2line(vector):
  text=""
  for v in vector:
     text+=model_wv.most_similar( [ v ], [], 1)[0][0]
  return text
  
def inflation(vector):
  vector+=np.random.randn(*vector.shape)*0.1
  return vector

In [0]:
q=[]
a=[]
with open("talk.txt") as fp:
  for line in fp:
    name,text=line.split("\t")
    if name=="":#個人情報保護の観点から削除しています
      a.append(line2vec(text,length=20))
    else:
      q.append(line2vec(text,length=20)[::-1])
q=np.asarray(q)
a=np.asarray(a)

In [0]:
 q.shape

In [0]:
a.shape

In [0]:
q=q.reshape(-1,20,200)
a=a.reshape(-1,20,200)
q_origin=copy.deepcopy(q)
a_origin=copy.deepcopy(a)

for i in range(10):
  q=np.vstack((q,inflation(q_origin)))
  a=np.vstack((a,inflation(a_origin)))

In [0]:
model = AttentionSeq2Seq(input_dim=200,input_length=20, output_length=20, output_dim=200)

# 学習の設定
model.compile(loss='mse', optimizer='Adam')

In [0]:
model.fit(q, a, epochs=200, batch_size=128)
model.save("er.hdf5")

In [0]:
#model.load_weights("er.hdf5")

In [0]:
def talk(text):
  vector=np.asarray(line2vec(text,length=20)[::-1])
  vector=inflation(vector)
  vector=vector.reshape(1,20,200)
  predict=model.predict(vector,batch_size=132)
  return vec2line(predict.reshape(20,200))

In [0]:
talk("元気?")